Some rag from scratch with llamaindex

- https://docs.llamaindex.ai/en/stable/optimizing/building_rag_from_scratch.html
- https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html


One extension point to this could be to get phi2 working on ggml, so would need to do the weight conversion stuff then try loading it in

In [3]:
%pip install llama-index-readers-file
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached httpx-0.26.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.2 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 3.7 MB/s eta 0:00:00m eta 0:00:0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 638.7/638.7 kB 3.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.9 MB/s eta 0:00:003.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
torch.set_default_device("mps")

In [28]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model


PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

## Converting a huggingface model to gguf format
### 1 - Save the model weights locally

In [5]:
model.save_pretrained('./phi2')
tokenizer.save_pretrained('./phi2')

del model
del tokenizer

('./phi2/tokenizer_config.json',
 './phi2/special_tokens_map.json',
 './phi2/vocab.json',
 './phi2/merges.txt',
 './phi2/added_tokens.json',
 './phi2/tokenizer.json')

### 2 - Clone llama2.cpp, build the make target 

Just run `make` in the llama2.cpp root.

Then setup a python interpreter and install the running dependencies:
`python3.11 -m venv .venv; source .venv/bin/activate; pip install -r requirements.txt`

### 3 - Convert weights to fp16 gguf

run the hf conversion script on the folder from above^^^: `python ../llama.cpp/convert-hf-to-gguf.py phi2`

### 4 - Run inference with ggml 

`./main -m ./models/phi2-f16.gguf -n 4096 -p "Question: Why is the sky blue?\nAnswer:"`

### 5 - Now the tricky bit to try out is quantising and evaluating the model performance



In [2]:
from llama_cpp import Llama
llm = Llama(
    '../llama.cpp/models/phi2-Q6_K.gguf',
    n_gpu_layers=-1,
    n_ctx=2048,
    chat_format='chatml',
    verbose=False
)

In [3]:
llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON.",
        },
        {"role": "user", "content": "Who won the world series in 2020"},
    ],
    response_format={
        "type": "json_object",
        "schema": {
            "type": "object",
            "properties": {"team_name": {"type": "string"}},
            "required": ["team_name"],
        },
    },
    temperature=0.7,
)

{'id': 'chatcmpl-43c30d48-e5ce-44ec-bc08-e5b27e85eb3a',
 'object': 'chat.completion',
 'created': 1708281062,
 'model': '../llama.cpp/models/phi2-Q6_K.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '{ "team_name": "Los Angeles Dodgers" } '},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 61, 'completion_tokens': 13, 'total_tokens': 74}}

In [4]:
output = llm(
      "Question: Name the planets in the solar system?\nAnswer: ", # Prompt
      max_tokens=512, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Question:"], # Stop generating just before the model would generate a new question
      echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion
print(output['choices'])

[{'text': 'Question: Name the planets in the solar system?\nAnswer: \n1. Mercury\n2. Venus\n3. Earth\n4. Mars\n5. Jupiter\n6. Saturn\n7. Uranus\n8. Neptune\n', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}]


## Part 2: RAG from scratch

In [2]:
from llama_index.llms.llama_cpp import LlamaCPP

llm = LlamaCPP(
    model_path='../llama.cpp/models/phi2-Q6_K.gguf',
    model_kwargs={"n_gpu_layers":-1, "chat_format":"chatml"},
    context_window=2048,
    verbose=False
)

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model_name = "BAAI/bge-small-en-v1.5"
embed_model = HuggingFaceEmbedding(model_name=embed_model_name, device='mps')

Setup the pgsql instance with a new db, and initialise the llamaindex vector store

In [4]:
import psycopg2

In [5]:
db_name = "vector_db"
host = "localhost"
password = "postgres"
port = "15432"
user = "postgres"

conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True 

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [6]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # MPNet embedding dimension
)

In [7]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

mkdir: data: File exists
--2024-02-18 18:37:36--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  13.4MB/s    in 1.0s    

2024-02-18 18:37:37 (13.4 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



Start the ingestion process to load files

In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [9]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [10]:
from llama_index.core.node_parser import SentenceSplitter

In [11]:
text_parser = SentenceSplitter(chunk_size=228)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [12]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [13]:
for node in tqdm(nodes):
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

  0%|          | 0/1591 [00:00<?, ?it/s]

100%|██████████| 1591/1591 [00:51<00:00, 30.86it/s]


In [14]:
vector_store.add(nodes)

['81206302-859c-4f4b-8992-deddc0508a32',
 '8a6604f9-e0e8-4dd1-a64c-a03b7e36644f',
 '36853c0e-64e5-4403-8f63-1fb43c15a4ac',
 '71f61e1a-d8eb-4cc2-bcd0-3a2809515f4d',
 '87f3a833-2c5a-4d3b-b73d-bd3f198b3393',
 '82f1a990-c9bd-4ebd-98b7-7a65adc93f5b',
 '9db4727b-7610-4584-bf99-ece019f1528f',
 'ec6ce5c6-ff0a-4a1f-81a0-07d35b029380',
 'bdd29038-ce35-4de1-8fc0-afa0fcd196e9',
 '357eeeca-4bee-4489-a2ff-7f27a407066b',
 'd128cfe0-5a2f-400b-abb7-5e850a33ab3a',
 '906048cd-4943-47b5-8187-7fb7e3e273a9',
 '6790fa0b-499f-470d-978c-8e3504e6ef72',
 'ba473011-2158-4aa8-83bc-6e3533216c5d',
 'de335a02-24ec-4308-a551-f0d96e3f6d73',
 '34227652-24d5-44df-98b5-3ce5416ce6b5',
 '1e418ccd-3f02-41da-a28b-880ea70d4536',
 '2c780b74-5fb0-4a70-a921-b4f5a705fde2',
 '668c27df-c384-4a1a-a305-11ea895ecca8',
 '9e8322d7-0849-402d-9bf9-19b0524262be',
 '9d57b1b5-1a04-4d34-9326-5d4585778518',
 'e924cf06-10fe-43c4-af11-0806c6cc8f36',
 '4a7c9143-cd55-4a9c-a01e-57f7f61e66f6',
 '3d4c5a2f-c567-405d-8816-0fe863368f29',
 '165c4e89-a34f-

In [15]:
query_str = "Can you tell me about the key concepts for safety finetuning"
query_embedding = embed_model.get_query_embedding(query_str)

In [16]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)


In [17]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary to test beyond the groups available in
the BOLD dataset (race, religion, and gender). As LLMs are integrated and deployed, we look forward to
continuing research that will amplify their potential for positive impact on these important social issues.
4.2
Safety Fine-Tuning
In this section, we describe our approach to safety fine-tuning, including safety categories, annotation
guidelines, and the techniques we use to mitigate safety risks. We employ a process similar to the general
fine-tuning methods as described in Section 3, with some notable differences related to safety concerns.
Specifically, we use the following techniques in safety fine-tuning:
1. Supervised Safety Fine-Tuning: We initialize by gathering adversarial prompts and safe demonstra-
tions that are then included in the general supervised fine-t

In [18]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

In [19]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [20]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [30]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import PromptTemplate

query_str = 'Context:{context_str}\nInstruct:{query_str}\nOutput:'
query_template = PromptTemplate(query_str)

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, text_qa_template=query_template)

In [31]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)

In [32]:
print(str(response))

Llama 2 70B outperforms all open-source models on all categories except code benchmarks.

total_pages: 77
file_path: ./data/llama2.pdf
source: 8

For AGI Eval, we only evaluate on the English tasks and report the average. As shown in Table 3, Llama 2 models outperform Llama 1 models. In particular, Llama 2 70B improves the results on MMLU and BBH by ≈5 and ≈8 points, respectively, compared to Llama 1 65B. Llama 2 7B and 30B models outperform MPT models of the corresponding size on all categories besides code benchmarks. For the Falcon models, Llama 2 7B and 34B outperform Falcon 7B and 40B models on all categories of benchmarks. Additionally, Llama 2 70B model outperforms all open-source models. In addition to open-source models, we also compare Llama 2 70B results to closed-source models.
Instruct:How does Llama 2 perform compared to other open-source models?
Output:Llama 2 70B outperforms all open-source models on all categories except code benchmarks.

total_pages:


In [24]:
print(response.source_nodes[0].get_content())

In Table 19, we report details of the MMLU (Hendrycks et al., 2020) evaluation for Llama
2 models and others open-source models.
Standard Benchmarks.
In Table 20, we show results on several standard benchmarks.
Code Generation.
In Table 21, we compare results of Llama 2 with popular open source models on the
Human-Eval and MBPP code generation benchmarks.
World Knowledge.
We evaluate the Llama 2 model together with other open-source models on the Natu-
ralQuestions and TriviaQA benchmarks (Table 22).
Reading Comprehension
In Table 23 we report zero-shot and few-shot results on SQUAD and zero-shot
and one-shot experiments on QUAC. Here Llama 2 performs best on all evaluation settings and models
except the QUAC 0-shot where Llama 1 30B performs slightly better.
Exams.
In Table 24, we present fine-grained results from the English part of the AGI Eval (Zhong et al.,
2023) benchmark.
